### Loading Imports

In [1]:
import pandas as pd
import numpy as np
import re
import spacy
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, MaxPooling1D, Dropout, Activation
#from keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Loading Data

In [2]:
atis_1_train = pd.read_csv("dados/Atis_1/atis.train.csv")
atis_1_val = pd.read_csv("dados/Atis_1/atis.dev.csv")
atis_1_test = pd.read_csv("dados/Atis_1/atis.test.csv")

atis_2_train = pd.read_csv("dados/Atis_2/atis_intents_train.csv")
atis_2_test = pd.read_csv("dados/Atis_2/atis_intents_test.csv")

In [3]:
print("Atis 1: ",atis_1_train.shape)
print("Atis 1: ",atis_1_val.shape)
print("Atis 1: ",atis_1_test.shape)
print("##############################")
print("Atis 2: ",atis_2_train.shape)
print("Atis 2: ",atis_2_test.shape)

Atis 1:  (4274, 4)
Atis 1:  (572, 4)
Atis 1:  (586, 4)
##############################
Atis 2:  (4833, 2)
Atis 2:  (799, 2)


### Pre processamento dos Dados
* Dataset a ser utilizado: <strong>Atis_1</strong>
* Verificar  <strong>Intents</strong>
* Verificar se existe algum <strong>valor Nan</strong>
* Escolher colunas com <strong>Tokens(sentenças) e Intents</strong>
* <strong>Eliminar</strong> caracter <strong>BOS</strong> do inicio das sentenças
* <strong>Eliminar</strong> caracter <strong>EOS</strong> do final das sentenças

In [4]:
atis_1_train.head(3)

,id,tokens,slots,intent
0,train-00001,BOS what is the cost of a round trip flight fr...,O O O O O O O B-round_trip I-round_trip O O B-...,atis_airfare
1,train-00002,BOS now i need a flight leaving fort worth and...,O O O O O O O B-fromloc.city_name I-fromloc.ci...,atis_flight
2,train-00003,BOS i need to fly from kansas city to chicago ...,O O O O O O B-fromloc.city_name I-fromloc.city...,atis_flight


In [5]:
atis_1_train['intent'].value_counts()

atis_flight                 3173
atis_airfare                 351
atis_ground_service          207
atis_airline                 139
atis_abbreviation             94
atis_aircraft                 70
atis_quantity                 41
atis_flight_time              38
atis_capacity                 29
atis_distance                 24
atis_airport                  23
atis_flight#atis_airfare      21
atis_city                     18
atis_ground_fare              18
atis_flight_no                16
atis_meal                      9
atis_restriction               3
Name: intent, dtype: int64

In [6]:
print("Valores Nan em train:\n",atis_1_train.isnull().sum())
print("#########################################################")
print("Valores Nan em val:\n",atis_1_val.isnull().sum())
print("#########################################################")
print("Valores Nan em test:\n",atis_1_test.isnull().sum())

Valores Nan em train:
 id        0
tokens    0
slots     0
intent    0
dtype: int64
#########################################################
Valores Nan em val:
 id        0
tokens    0
slots     0
intent    0
dtype: int64
#########################################################
Valores Nan em test:
 id        0
tokens    0
slots     0
intent    0
dtype: int64


In [7]:
atis_1_train = atis_1_train.loc[:,['tokens','intent']]
atis_1_val = atis_1_val.loc[:,['tokens','intent']]
atis_1_test = atis_1_test.loc[:,['tokens','intent']]

In [8]:
atis_1_train.head(3)

,tokens,intent
0,BOS what is the cost of a round trip flight fr...,atis_airfare
1,BOS now i need a flight leaving fort worth and...,atis_flight
2,BOS i need to fly from kansas city to chicago ...,atis_flight


In [9]:
# eliminando caracter indesejado do inicio BOS e final EOS sentenças
atis_1_train['tokens'] = [x.replace('BOS ','') for x in atis_1_train['tokens']]
atis_1_val['tokens'] = [x.replace('BOS ','') for x in atis_1_val['tokens']]
atis_1_test['tokens'] = [x.replace('BOS ','') for x in atis_1_test['tokens']]

atis_1_train['tokens'] = [x.replace(' EOS','') for x in atis_1_train['tokens']]
atis_1_val['tokens'] = [x.replace(' EOS','') for x in atis_1_val['tokens']]
atis_1_test['tokens'] = [x.replace(' EOS','') for x in atis_1_test['tokens']]

In [10]:
dict_Labels = {}
count = 0
for intent in atis_1_train['intent']:
    if intent not in dict_Labels:
        dict_Labels[intent] = count
        count += 1

In [11]:
print(dict_Labels)

{'atis_airfare': 0, 'atis_flight': 1, 'atis_abbreviation': 2, 'atis_ground_service': 3, 'atis_restriction': 4, 'atis_airport': 5, 'atis_quantity': 6, 'atis_meal': 7, 'atis_airline': 8, 'atis_city': 9, 'atis_flight_no': 10, 'atis_ground_fare': 11, 'atis_flight_time': 12, 'atis_flight#atis_airfare': 13, 'atis_distance': 14, 'atis_aircraft': 15, 'atis_capacity': 16}


In [12]:
atis_1_train['Label_intent'] = [dict_Labels[x] for x in atis_1_train['intent']]
atis_1_val['Label_intent'] = [dict_Labels[x] for x in atis_1_val['intent']]
atis_1_test['Label_intent'] = [dict_Labels[x] for x in atis_1_test['intent']]

In [13]:
atis_1_train.head(10)

,tokens,intent,Label_intent
0,what is the cost of a round trip flight from p...,atis_airfare,0
1,now i need a flight leaving fort worth and arr...,atis_flight,1
2,i need to fly from kansas city to chicago leav...,atis_flight,1
3,what is the meaning of meal code s,atis_abbreviation,2
4,show me all flights from denver to pittsburgh ...,atis_flight,1
5,show me all us air flights from atlanta to den...,atis_flight,1
6,list the nonstop flights early tuesday morning...,atis_flight,1
7,show me the flights from st. petersburg to tor...,atis_flight,1
8,i need a listing of flights from new york city...,atis_flight,1
9,show me the flights on american airlines which...,atis_flight,1


### Codificação

In [14]:
atis_1_train_label_intent = atis_1_train['Label_intent']
atis_1_train_label_intent = to_categorical(np.asarray(atis_1_train_label_intent))
atis_1_val_label_intent = atis_1_val['Label_intent']
atis_1_val_label_intent = to_categorical(np.asarray(atis_1_val_label_intent))
print(atis_1_train_label_intent.shape)
print(atis_1_val_label_intent.shape)

(4274, 17)
(572, 17)


In [15]:
# Usar apenas as sentencas de treinameto
# OOV para out of vocab
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(atis_1_train['tokens'])

word_index = tokenizer.word_index
print(word_index)

{'<OOV>': 1, 'to': 2, 'from': 3, 'flights': 4, 'the': 5, 'on': 6, 'flight': 7, 'me': 8, 'what': 9, 'show': 10, 'san': 11, 'boston': 12, 'a': 13, 'denver': 14, 'in': 15, 'francisco': 16, 'and': 17, 'i': 18, 'atlanta': 19, 'is': 20, 'dallas': 21, 'pittsburgh': 22, 'all': 23, 'list': 24, 'baltimore': 25, 'philadelphia': 26, 'like': 27, 'are': 28, 'airlines': 29, 'that': 30, 'washington': 31, 'of': 32, 'between': 33, 'please': 34, 'morning': 35, 'leaving': 36, 'pm': 37, 'would': 38, 'fly': 39, 'city': 40, 'for': 41, 'need': 42, 'fare': 43, 'wednesday': 44, 'first': 45, 'after': 46, 'there': 47, 'oakland': 48, 'trip': 49, 'ground': 50, 'arriving': 51, 'you': 52, 'cheapest': 53, 'which': 54, 'round': 55, 'transportation': 56, "i'd": 57, 'before': 58, 'does': 59, 'class': 60, 'with': 61, 'available': 62, 'st': 63, 'afternoon': 64, 'one': 65, 'milwaukee': 66, 'new': 67, 'have': 68, 'at': 69, 'way': 70, 'fares': 71, 'american': 72, 'dc': 73, 'give': 74, 'york': 75, 'leave': 76, 'thursday': 77, 

In [16]:
def valor_maior_sentenca():
    maior_sentenca_train = atis_1_train['tokens'].str.len().max()
    maior_sentenca_val = atis_1_val['tokens'].str.len().max()
    maior_sentenca_test = atis_1_test['tokens'].str.len().max()
    maior_sentenca = max([maior_sentenca_train,maior_sentenca_test,maior_sentenca_val])
    return maior_sentenca

# tamanho do meu vocabulario voca_size adn 1 to unknow
vocab_size = len(word_index) + 1

# tamanho da maior sentenca dos dados originais
maior_sentenca = valor_maior_sentenca()

print("Tamanho do vocab: ",vocab_size," Tamanho da Sentenca: ",maior_sentenca)

Tamanho do vocab:  874  Tamanho da Sentenca:  259


In [17]:
# Transformar as sentencas dos dados de treinamento e teste para sequencias numericas. ex [[1,2,6],[65,952,15,65],[1,2]]
train_data_sentences = tokenizer.texts_to_sequences(atis_1_train['tokens'])
val_data_sentences = tokenizer.texts_to_sequences(atis_1_val['tokens'])
test_data_sentences = tokenizer.texts_to_sequences(atis_1_test['tokens'])

# normalizar o tamnho das sequencias usando padding. ex [[0,1,2,6],[65,952,15,65],[0,0,1,2]]
train_padded = pad_sequences(train_data_sentences, maxlen=maior_sentenca)
val_padded = pad_sequences(val_data_sentences, maxlen=maior_sentenca)
test_padded = pad_sequences(test_data_sentences, maxlen=maior_sentenca)

print("Conjunto de treinamento: ","\n",train_padded[0:19])
print("Conjunto de validação: ","\n",val_padded[0:19])
print("Conjunto de teste: ","\n",test_padded[0:19])

Conjunto de treinamento:  
 [[  0   0   0 ...   6 168 252]
 [  0   0   0 ...  37 118  79]
 [  0   0   0 ...   5 459 220]
 ...
 [  0   0   0 ...  12  17  48]
 [  0   0   0 ...  32 607 194]
 [  0   0   0 ... 132 305 608]]
Conjunto de validação:  
 [[  0   0   0 ... 350 168 252]
 [  0   0   0 ... 200 263  37]
 [  0   0   0 ...   5 220  46]
 ...
 [  0   0   0 ...  58 191  37]
 [  0   0   0 ...  15  31  73]
 [  0   0   0 ...  25   2  21]]
Conjunto de teste:  
 [[  0   0   0 ... 350 119   1]
 [  0   0   0 ... 200 263  37]
 [  0   0   0 ...   5 459 220]
 ...
 [  0   0   0 ...   2  90 367]
 [  0   0   0 ...  15  31  73]
 [  0   0   0 ...  53 174 258]]


In [18]:
## Load the spacy model: nlp
# nlp = spacy.load('en_core_web_md')
#
##tamnho do vocab
#print(len(nlp.vocab.strings))
#
## dimension vector
#print(nlp.vocab.vectors_length)

In [19]:
#def valor_maior_sentenca():
#    maior_sentenca_train = atis_1_train['tokens'].str.len().max()
#    maior_sentenca_val = atis_1_val['tokens'].str.len().max()
#    maior_sentenca_test = atis_1_test['tokens'].str.len().max()
#    maior_sentenca = max([maior_sentenca_train,maior_sentenca_test,maior_sentenca_val])
#    return maior_sentenca

In [20]:
#count = 0
#for key, vector in nlp.vocab.vectors.items():
#    if nlp.vocab.strings[key] != None:
#        count += 1
#    #print(key, nlp.vocab.strings[key], vector[:3])

In [21]:
## valores para o modelo deep learning
#
##vocab_size = len(nlp.vocab.strings)
#vocab_size = count + 1
#
## tamanho da maior sentença - input_length
#maior_sentenca = valor_maior_sentenca()
#
## embedding
#embedding_dim = nlp.vocab.vectors_length
#print(vocab_size)
#print(embedding_dim)
#

In [22]:
# guardar valor de encoding dos intents
#def vetorizar_sentencas(sentencas,intents):
#    # Calculate the length of sentences
#    n_sentences = len(sentencas)
#
#    # Calculate the dimensionality of nlp
#    embedding_dim = nlp.vocab.vectors_length
#
#    # Initialize the array with zeros: X
#    sentencas_vetorizadas = np.zeros((n_sentences, embedding_dim))
#
#    # Iterate over the sentences
#    for idx, sentence in enumerate(sentencas):
#        # Pass each sentence to the nlp object to create a document
#        doc = nlp(sentence)
#        # Save the document's .vector attribute to the corresponding row in X
#        sentencas_vetorizadas[idx, :] = doc.vector
#
#    # encoding intents
#    count = 0
#    # Iterate over the intents
#    for intent_label in intents:
#        if(intent_label not in dict_Labels):
#            dict_Labels[intent_label] = count
#            count += 1
#
#
#    intents_encode = np.array([dict_Labels[x] for x in intents])
#    
#    return [sentencas_vetorizadas,intents_encode]
#
#train_vectors = vetorizar_sentencas(atis_1_train["tokens"],atis_1_train['intent'])
#val_vectors = vetorizar_sentencas(atis_1_val["tokens"],atis_1_val['intent'])
#test_vectors = vetorizar_sentencas(atis_1_test["tokens"],atis_1_test['intent'])
#
##tf.keras.utils.to_categorical
#train_sentencas_vetorizadas = train_vectors[0]
#train_intent_label = to_categorical(train_vectors[1])
#
#val_sentencas_vetorizadas = val_vectors[0]
#val_intent_label = to_categorical(val_vectors[1])
#
#test_sentencas_vetorizadas = test_vectors[0]
#test_intent_label = to_categorical(test_vectors[1])
#
#print("Shape: ",train_sentencas_vetorizadas.shape," type: ",train_sentencas_vetorizadas.dtype)
#print("Shape: ",train_intent_label.shape," type: ",train_intent_label.dtype)
#print("Shape: ",val_sentencas_vetorizadas.shape," type: ",val_sentencas_vetorizadas.dtype)
#print("Shape: ",val_intent_label.shape," type: ",val_intent_label.dtype)
#print("Shape: ",test_sentencas_vetorizadas.shape," type: ",test_sentencas_vetorizadas.dtype)
#print("Shape: ",test_intent_label.shape," type: ",test_intent_label.dtype)

#### Model

In [23]:
embedding_dim = maior_sentenca

In [24]:
model1 = Sequential()
model1.add(Embedding(vocab_size,embedding_dim, input_length=maior_sentenca))
model1.add(GlobalAveragePooling1D())
model1.add(Dense(150, activation='sigmoid'))
model1.add(Dropout(0.25))
model1.add(Dense(50, activation='relu'))
#model1.add(Dense(25, activation='relu'))
model1.add(Dense(17, activation='softmax'))


#model.compile(loss='categorical_crossentropy', optimizer=sgd)
model1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 259, 259)          226366    
_________________________________________________________________
global_average_pooling1d (Gl (None, 259)               0         
_________________________________________________________________
dense (Dense)                (None, 150)               39000     
_________________________________________________________________
dropout (Dropout)            (None, 150)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                7550      
_________________________________________________________________
dense_2 (Dense)              (None, 17)                867       
Total params: 273,783
Trainable params: 273,783
Non-trainable params: 0
__________________________________________________

In [26]:
num_epochs = 50
history1 = model1.fit(train_padded, atis_1_train_label_intent,
                      validation_data=(val_padded, atis_1_val_label_intent),
                      epochs=num_epochs,
                      verbose=1)

Epoch 1/50
134/134 [==============================] - 5s 38ms/step - loss: 1.3018 - accuracy: 0.7052 - val_loss: 1.1457 - val_accuracy: 0.7395
Epoch 2/50
134/134 [==============================] - 4s 33ms/step - loss: 1.1441 - accuracy: 0.7424 - val_loss: 1.1376 - val_accuracy: 0.7395
Epoch 3/50
134/134 [==============================] - 6s 42ms/step - loss: 1.1283 - accuracy: 0.7424 - val_loss: 1.1204 - val_accuracy: 0.7395
Epoch 4/50
134/134 [==============================] - 5s 38ms/step - loss: 1.0910 - accuracy: 0.7424 - val_loss: 1.0516 - val_accuracy: 0.7395
Epoch 5/50
134/134 [==============================] - 5s 39ms/step - loss: 0.9869 - accuracy: 0.7424 - val_loss: 0.9444 - val_accuracy: 0.7395
Epoch 6/50
134/134 [==============================] - 6s 44ms/step - loss: 0.8732 - accuracy: 0.7532 - val_loss: 0.8893 - val_accuracy: 0.7535
Epoch 7/50
134/134 [==============================] - 5s 41ms/step - loss: 0.7925 - accuracy: 0.7824 - val_loss: 0.7933 - val_accuracy: 0.7727

#### Evaluate

In [27]:
model1.evaluate(train_padded,atis_1_train_label_intent)

134/134 [==============================] - 2s 19ms/step - loss: 0.2145 - accuracy: 0.9378


[0.2144516110420227, 0.9377632141113281]

In [28]:
model1.evaluate(val_padded,atis_1_val_label_intent)

18/18 [==============================] - 0s 22ms/step - loss: 0.4509 - accuracy: 0.9091


[0.4509293735027313, 0.9090909361839294]

In [29]:
atis_1_test_label_intent = atis_1_test['Label_intent']
atis_1_test_label_intent = to_categorical(np.asarray(atis_1_test_label_intent))

model1.evaluate(test_padded,atis_1_test_label_intent)

19/19 [==============================] - 0s 19ms/step - loss: 0.3694 - accuracy: 0.8942


[0.36935991048812866, 0.894197940826416]

#### Saving
* tokenizer
* model

In [35]:
import pickle
# saving
with open('recursos/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [37]:
#from keras.models import load_model
#
#model1.save('../recursos/my_model.h5') 

In [38]:
#filepath = 'recursos/my_model.h5'
#
#model1.save(model1, filepath)
#
#tf.keras.models.save_model(
#    model1, filepath , overwrite=True, include_optimizer=True, save_format=None,
#    signatures=None
#)
##https://www.tensorflow.org/api_docs/python/tf/keras/models/save_model

In [39]:
#predicao =  model1.predict(test_padded)
#count = 1
#for elemento in predicao[:10]:
#    print("Sentença {} ".format(count),"\nPredição {}: ".format(count),elemento)
#    print("Best value of confidence: {}".format(np.argmax(elemento)))
#    print("Intent da Sentença: {}".format(inv_dict_labels[np.argmax(elemento)]))
#    print("\n")
#    count += 1

#### Avaliando modelo com conjunto de Teste

In [40]:
# Função pare test
def Predicao_Labels(sentenca_codificada):
    # inverter o dicionario dict_labels para retornar o intent pelo seu valor chave inteiro
    inv_dict_labels = {v: k for k, v in dict_Labels.items()}
    #print(inv_dict_labels)
    # passar a sentenca codificada
    predicao = model1.predict([sentenca_codificada])
    #print(predicao)
    
    intent_value = [inv_dict_labels[np.argmax(x)] for x in predicao]
    
    return intent_value

# lista com valores da predicao
Predicao_Labels(test_padded)[:5]

['atis_flight',
 'atis_flight',
 'atis_flight',
 'atis_abbreviation',
 'atis_flight']

In [41]:
test_labels = np.array(atis_1_test['intent'])
# Acurácia nos dados de test
from sklearn.metrics import accuracy_score # importando a biblioteca para calcular a acurácia
acuracia = accuracy_score(test_labels,np.array(Predicao_Labels(test_padded)))
print('Acurácia: %f' % acuracia)

Acurácia: 0.894198


In [71]:
from sklearn.metrics import classification_report
print(classification_report(test_labels,np.array(Predicao_Labels(test_padded))))

                          precision    recall  f1-score   support

       atis_abbreviation       0.71      0.62      0.67        16
           atis_aircraft       0.21      0.50      0.30         8
            atis_airfare       0.82      0.93      0.87        54
            atis_airline       0.48      0.56      0.51        18
            atis_airport       0.00      0.00      0.00         4
           atis_capacity       0.50      0.75      0.60         4
               atis_city       0.00      0.00      0.00         3
           atis_distance       1.00      0.67      0.80         3
             atis_flight       0.97      0.97      0.97       424
atis_flight#atis_airfare       0.00      0.00      0.00         3
          atis_flight_no       0.00      0.00      0.00         2
        atis_flight_time       0.33      0.14      0.20         7
        atis_ground_fare       0.67      0.50      0.57         4
     atis_ground_service       0.96      0.93      0.95        29
         

C:\Users\Usuário\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Usuário\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Usuário\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

#### Get Intent de Senteça

In [43]:
# intent de sentenças
def Intent(sentenca,modelo):
    
    # loading model and tokenizer
    with open('recursos/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
        
    # load model not working
    #filepath = '../recursos/my_model.h5'
    #model = tf.saved_model.load(filepath)
    
    # codificar a sentenca

    sentenca = [sentenca]
    sentenca_encoded = tokenizer.texts_to_sequences(sentenca)
    sentenca_padded = pad_sequences(sentenca_encoded)
    #print(sentenca_padded)
    #intent = Predicao_Labels(sentenca_padded)
    
    #Inverter dict_labels para poder pegar o intent 
    inv_dict_labels = {v: k for k, v in dict_Labels.items()}
    
    # Predicao de intent apartir da sentenca codificada e o modelo
    predicao = modelo.predict(sentenca_padded)
    intent_value = [inv_dict_labels[np.argmax(x)] for x in predicao]
    
    return intent_value 

In [45]:
sentenca1='I want a list of the flights from Boston to DC'
sentenca2='Are there any airports nearby?'
sentenca3='fares from DC to NY'
print(Intent(sentenca1,model1))
print(Intent(sentenca2,model1))
print(Intent(sentenca3,model1))

['atis_flight']
['atis_ground_service']
['atis_airfare']


### Intent Respostas

In [49]:
# criar dicionarios para guardar as respostas por intent.
# criar alguns intents basicos greetings...

#templates
user_template = "USER : {0}" 
bot_template = "BOT : {0}"

dict_atis = dict_Labels

# keywords para identificar padrões de intents basicos
keywords = {'greet': ['hello', 'hi', 'hey'],
            'goodbye': ['bye', 'farewell'],
            'thankyou': ['thank', 'thx']}

# intents são chaves do dicionario e o valor são as respostas
responses = {'default': 'default message',
             'goodbye': 'goodbye for now',
             'greet': 'Hello you! :)',
             'thankyou': 'you are very welcome'}

# Novo dicionario de respostas do bot
update_responses = {}

for intent, value in dict_Labels.items():
    if (intent not in update_responses):
        update_responses[intent] = []

for intent,resposta in responses.items():
    if intent not in update_responses:
        update_responses[intent] = [resposta]

# Define a dictionary of patterns
patterns = {}

# Iterate over the keywords dictionary
for intent, keys in keywords.items():
    
    # Create regular expressions and compile them into pattern objects
    patterns[intent] = re.compile('|'.join(keys))
    
# Print the dicionarios
print(patterns)
print(responses)
print(update_responses)

{'greet': re.compile('hello|hi|hey'), 'goodbye': re.compile('bye|farewell'), 'thankyou': re.compile('thank|thx')}
{'default': 'default message', 'goodbye': 'goodbye for now', 'greet': 'Hello you! :)', 'thankyou': 'you are very welcome'}
{'atis_airfare': [], 'atis_flight': [], 'atis_abbreviation': [], 'atis_ground_service': [], 'atis_restriction': [], 'atis_airport': [], 'atis_quantity': [], 'atis_meal': [], 'atis_airline': [], 'atis_city': [], 'atis_flight_no': [], 'atis_ground_fare': [], 'atis_flight_time': [], 'atis_flight#atis_airfare': [], 'atis_distance': [], 'atis_aircraft': [], 'atis_capacity': [], 'default': ['default message'], 'goodbye': ['goodbye for now'], 'greet': ['Hello you! :)'], 'thankyou': ['you are very welcome']}


In [50]:
import random
# Define a function to find the intent of a message
def match_intent(message):
    matched_intent = None
    
    # para intents dentro do dicionario de padões
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if re.search(pattern, message) is not None:
            matched_intent = intent

    return matched_intent

# Define a respond function
def respond(message):
    
    intent = None
    # Call the match_intent function
    if(match_intent(message) is not None):
        intent = match_intent(message)
    else:
        listaIntent = Intent(message,model1)
        intent = listaIntent[0]
        #print(intent)
        
    # Fall back to the default response
    key = "default"
    if intent in update_responses:
        if len(update_responses[intent]) != 0:
            key = intent
    resposta = random.choice(update_responses[key])
    
    return resposta

# Define a function that sends a message to the bot: send_message
def send_message(message):
    # Print user_template including the user_message
    print(user_template.format(message))
    # Get the bot's response to the message
    response = respond(message)
    # Print the bot template including the bot's response.
    print(bot_template.format(response))

# Send messages
send_message("hello!")

send_message("Flight from NY")

USER : hello!
BOT : Hello you! :)
USER : Flight from NY
BOT : default message


In [ ]:
def chatbot():
    while True:
        message = input("Send Message: ")
        if(message == 'exit'):
            break
        else:
            send_message(message)
chatbot()

### Entidades

In [51]:
sentence = atis_1_test['tokens'][0]
print(sentence)

what are the coach flights between dallas and baltimore leaving august tenth and returning august twelve


In [52]:
import spacy
nlp = spacy.load("en_core_web_md")
doc = nlp(sentence)

In [63]:
from spacy import displacy
displacy.render(doc,style='ent',jupyter=True)
displacy.render(doc,style='dep',jupyter=True,options={'distance':100})

In [64]:
def show_ents(doc):
    if doc.ents:
        for ent in doc.ents:
            print(ent.text+" - "+str(ent.start_char)+" - "+str(ent.end_char)+" - "+ent.label_+" - "+
                 str(spacy.explain(ent.label_)))
    else:
        print("No named entities found.")

In [65]:
show_ents(doc)

dallas - 35 - 41 - GPE - Countries, cities, states
baltimore - 46 - 55 - GPE - Countries, cities, states
august tenth - 64 - 76 - DATE - Absolute or relative dates or periods
august twelve - 91 - 104 - DATE - Absolute or relative dates or periods


In [72]:
import spacy
from spacy.lang.en import English
from spacy.pipeline import EntityRuler